In [44]:
import fastf1
import fastf1.plotting

from pathlib import Path
import pandas as pd
import numpy as np
import os  
from datetime import timedelta, datetime
import csv
from fastf1.ergast import Ergast

ergast = Ergast()

#parent_file = Path(__file__).resolve().parent.parent
parent_file = '/home/kurios/Documents/f1_analysis/'

year = 2025
race_quarter = 1
race_session = 'R'

#year = int(input("Year ? "))
#race_quarter = int(input("Race Quarter ? (1,4) "))
#race_session = input('Session ?  (S, R) ')

def race_quarter_value(race_quarter):
    match race_quarter:
        case 1 :
            return range(1, 7)
        case 2 :
            race_numbers = range(6, 13)
            return race_numbers
        case 3 :
            race_numbers = range(12, 19)
            return race_numbers
        case 4 :
            race_numbers = range(18, 25)
            return race_numbers

def parse_laptime_str(laptime_str):
    minutes, seconds = laptime_str.split(":")
    return float(minutes) * 60 + float(seconds)

In [160]:
race_years = list(race_quarter_value(race_quarter))
race_recaps = {}

i = 0
finished = False

while finished == False :
    try:
        iqr_driver_1 = 0
        iqr_driver_2 = 0

        avg_gap_driver_1 = 0
        avg_gap_driver_2 = 0

        lap_advantage_driver_1 = 0
        lap_advantage_driver_2 = 0

        pit_stop_time_driver_1 = 0
        pit_stop_time_driver_2 = 0

        pit_stop_number_driver_1 = 0
        pit_stop_number_driver_2 = 0

        finished_race_driver_1 = 0
        finished_race_driver_2 = 0

        for race_number in race_years:

            zero_time = timedelta(0)
            threshold_multiplier = 1.5

            keyword = f'{race_number}_{race_session}_race_info'
            for fname in os.listdir(parent_file + 'data/processed/'):
                if keyword in fname:
                    driver_data = fname
            os.chdir(parent_file + 'data/processed/')
            arr = pd.read_csv(driver_data)
            avg_seconds_driver_1 = parse_laptime_str(arr['avg_laptime_driver_1'][i])
            avg_seconds_driver_2 = parse_laptime_str(arr['avg_laptime_driver_2'][i])

            if (float(arr['iqr_driver_1'][i][:-2]) != float(0) 
                    and avg_seconds_driver_1 != timedelta(0) 
                        and avg_seconds_driver_2 != timedelta(0)
                            and avg_seconds_driver_1 <= threshold_multiplier * avg_seconds_driver_2
                                and avg_seconds_driver_2 <= threshold_multiplier * avg_seconds_driver_1):

                race_recaps[f'driver_recap_{i}'] = {}

                iqr_driver_1 += float(arr['iqr_driver_1'][i][:-2])
                
                if datetime.strptime(arr['avg_laptime_driver_1'][i], '%M:%S.%f') < datetime.strptime(arr['avg_laptime_driver_2'][i], '%M:%S.%f'):
                    one_race_gap = -round((pd.to_timedelta(zero_time - (datetime.strptime(arr['avg_laptime_driver_1'][i], '%M:%S.%f') - datetime.strptime(arr['avg_laptime_driver_2'][i], '%M:%S.%f'))).total_seconds()), 3)
                else : 
                    one_race_gap = round((pd.to_timedelta(datetime.strptime(arr['avg_laptime_driver_1'][i], '%M:%S.%f') - datetime.strptime(arr['avg_laptime_driver_2'][i], '%M:%S.%f')).total_seconds()), 3)
                
                avg_gap_driver_1 = avg_gap_driver_1 + one_race_gap

                lap_advantage_driver_1 +=(int(arr['fastest_driver_1'][i][:-3])/(int(arr['fastest_driver_1'][i][-2:]) - int(arr['safety_car_lap'][i][:-3]))*100)

                pit_stop_number_driver_1 += int(arr['number_pit_driver_1'][i])
                pit_stop_one_race_driver_1 = parse_laptime_str(arr['total_duration_pit_driver_1'][i][:-2])
                pit_stop_time_driver_1 = pit_stop_time_driver_1 + pit_stop_one_race_driver_1

                finished_race_driver_1 += 1

            if (float(arr['iqr_driver_2'][i][:-2]) != float(0) 
                    and avg_seconds_driver_1 != timedelta(0) 
                        and avg_seconds_driver_2 != timedelta(0)
                            and avg_seconds_driver_1 <= threshold_multiplier * avg_seconds_driver_2
                                and avg_seconds_driver_2 <= threshold_multiplier * avg_seconds_driver_1
            ):

                race_recaps[f'driver_recap_{i}_2'] = {}

                iqr_driver_2 += float(arr['iqr_driver_2'][i][:-2])

                if datetime.strptime(arr['avg_laptime_driver_1'][i], '%M:%S.%f') > datetime.strptime(arr['avg_laptime_driver_2'][i], '%M:%S.%f'):
                    one_race_gap = -round((pd.to_timedelta(zero_time - (datetime.strptime(arr['avg_laptime_driver_2'][i], '%M:%S.%f') - datetime.strptime(arr['avg_laptime_driver_1'][i], '%M:%S.%f'))).total_seconds()), 3)
                else : 
                    one_race_gap = round((pd.to_timedelta(datetime.strptime(arr['avg_laptime_driver_2'][i], '%M:%S.%f') - datetime.strptime(arr['avg_laptime_driver_1'][i], '%M:%S.%f')).total_seconds()), 3)
                
                avg_gap_driver_2 = avg_gap_driver_2 + one_race_gap

                lap_advantage_driver_2 +=(int(arr['fastest_driver_2'][i][:-3])/(int(arr['fastest_driver_2'][i][-2:]) - int(arr['safety_car_lap'][i][:-3]))*100)

                pit_stop_number_driver_2 += int(arr['number_pit_driver_2'][i])
                pit_stop_one_race_driver_2 = parse_laptime_str(arr['total_duration_pit_driver_2'][i][:-2])
                pit_stop_time_driver_2 = pit_stop_time_driver_2 + pit_stop_one_race_driver_2

                finished_race_driver_2 += 1     

        iqr_driver_1 = str(round(iqr_driver_1/finished_race_driver_1,3))
        iqr_driver_2 = str(round(iqr_driver_2/finished_race_driver_2,3))
        
        avg_gap_driver_1 = str(round(avg_gap_driver_1/finished_race_driver_1, 3))
        avg_gap_driver_2 = str(round(avg_gap_driver_2/finished_race_driver_2, 3))

        lap_advantage_driver_1 = str(round(lap_advantage_driver_1/finished_race_driver_1))
        lap_advantage_driver_2 = str(round(lap_advantage_driver_2/finished_race_driver_2))

        #print(pit_stop_time_driver_1)
        pit_stop_time_driver_1 = str(round(pit_stop_time_driver_1/pit_stop_number_driver_1, 3))
        pit_stop_time_driver_2 = str(round(pit_stop_time_driver_2/pit_stop_number_driver_2, 3))

        race_recaps[f'driver_recap_{i}'].update({'Driver Name' : arr['driver_1_name'][i], 'Average IQR' : iqr_driver_1 + 's', 'Average Gap Race Lap' : avg_gap_driver_1 + 's', 'Lap Advantage' : lap_advantage_driver_1 + '%', 'Average Pit Stop Time' : pit_stop_time_driver_1 + 's'})
        
        race_recaps[f'driver_recap_{i}_2'].update({'Driver Name' : arr['driver_2_name'][i], 'Average IQR' : iqr_driver_2 + 's', 'Average Gap Race Lap' : avg_gap_driver_2 + 's', 'Lap Advantage' : lap_advantage_driver_2 + '%', 'Average Pit Stop Time' : pit_stop_time_driver_2 + 's'})
        
        i += 1
    except:
        finished = True

In [161]:
print(race_recaps)

{'driver_recap_0': {'Driver Name': 'Max Verstappen', 'Average IQR': '2.003s', 'Average Gap Race Lap': '-0.663s', 'Lap Advantage': '83%', 'Average Pit Stop Time': '20.145s'}, 'driver_recap_0_2': {'Driver Name': 'Yuki Tsunoda', 'Average IQR': '2.119s', 'Average Gap Race Lap': '0.663s', 'Lap Advantage': '17%', 'Average Pit Stop Time': '21.294s'}, 'driver_recap_1': {'Driver Name': 'Pierre Gasly', 'Average IQR': '1.758s', 'Average Gap Race Lap': '-0.389s', 'Lap Advantage': '76%', 'Average Pit Stop Time': '24.793s'}, 'driver_recap_1_2': {'Driver Name': 'Jack Doohan', 'Average IQR': '1.402s', 'Average Gap Race Lap': '0.389s', 'Lap Advantage': '24%', 'Average Pit Stop Time': '24.015s'}, 'driver_recap_2': {'Driver Name': 'Kimi Antonelli', 'Average IQR': '1.907s', 'Average Gap Race Lap': '0.435s', 'Lap Advantage': '28%', 'Average Pit Stop Time': '23.692s'}, 'driver_recap_2_2': {'Driver Name': 'George Russell', 'Average IQR': '1.571s', 'Average Gap Race Lap': '-0.435s', 'Lap Advantage': '72%', 'A